In [1]:
import tkinter as tk
import heapq
import math
from collections import Counter

# Define compression_technique_var as a global variable
compression_technique_var = None

# Placeholder functions for compression techniques
def run_length_encoding(text):
    encoded_text = {}
    count = 1
    for i in range(1, len(text)):
        if text[i] == text[i - 1]:
            count += 1
        else:
            encoded_text[text[i - 1]] = count
            count = 1
    encoded_text[text[-1]] = count
    return encoded_text

# Function to perform Arithmetic encoding
def arithmetic_encoding(text):
    if not text:
        return {}

    # Calculate symbol frequencies
    freq_map = Counter(text)
    total_symbols = sum(freq_map.values())

    # Calculate cumulative probabilities
    cumulative_probabilities = {}
    lower_bound = 0
    for symbol, freq in freq_map.items():
        upper_bound = lower_bound + freq / total_symbols
        cumulative_probabilities[symbol] = (lower_bound, upper_bound)
        lower_bound = upper_bound

    # Initialize range
    low = 0.0
    high = 1.0
    precision = 32
    result = {}

    # Encode text using Arithmetic encoding
    for symbol in text:
        range_width = high - low
        low += range_width * cumulative_probabilities[symbol][0]
        high = low + range_width * (cumulative_probabilities[symbol][1] - cumulative_probabilities[symbol][0])

        while (high <= 0.5) or (low >= 0.5):
            if high <= 0.5:
                result[symbol] = result.get(symbol, "") + "0"
                low *= 2
                high *= 2
            else:
                result[symbol] = result.get(symbol, "") + "1"
                low = 2 * (low - 0.5)
                high = 2 * (high - 0.5)

    # Pad result with zeroes
    for symbol in result:
        result[symbol] += "0" * (precision - len(result[symbol]))

    return result

def golomb_encoding(text, b=3):
    encoded_text = {}
    for char in text:
        value = ord(char)
        quotient = value // (2 ** b)
        remainder = value % (2 ** b)
        encoded_text[char] = "1" * quotient + "0" + format(remainder, f'0{b}b')
    return encoded_text

# Calculate compression ratio for each technique
def calculate_compression_ratio(text, encoded_text):
    original_size = len(text) * 8  # Assuming each character is 8 bits
    encoded_size = len(encoded_text)
    ratio = (encoded_size / original_size) * 100
    return ratio

def lzw_encoding(text):
    dictionary = {chr(i): i for i in range(256)}
    next_code = 256
    result = {}
    current_sequence = ""
    for char in text:
        if current_sequence + char in dictionary:
            current_sequence += char
        else:
            result[current_sequence] = dictionary[current_sequence]
            dictionary[current_sequence + char] = next_code
            next_code += 1
            current_sequence = char
    if current_sequence:
        result[current_sequence] = dictionary[current_sequence]
    return result

# Define the Node class for Huffman tree
class Node:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

# Function to build Huffman tree
def build_huffman_tree(text):
    freq_map = Counter(text)
    priority_queue = [Node(char, freq) for char, freq in freq_map.items()]
    heapq.heapify(priority_queue)

    while len(priority_queue) > 1:
        left = heapq.heappop(priority_queue)
        right = heapq.heappop(priority_queue)
        merged = Node(None, left.freq + right.freq)
        merged.left = left
        merged.right = right
        heapq.heappush(priority_queue, merged)

    return priority_queue[0]

# Function to generate Huffman codes
def generate_huffman_codes(root, current_code="", codes={}):
    if root is None:
        return

    if root.char is not None:
        codes[root.char] = current_code
        return

    generate_huffman_codes(root.left, current_code + "0", codes)
    generate_huffman_codes(root.right, current_code + "1", codes)

# Function to perform Huffman encoding
def huffman_encoding(text):
    if not text:
        return ""

    # Build Huffman tree
    root = build_huffman_tree(text)

    # Generate Huffman codes
    codes = {}
    generate_huffman_codes(root, "", codes)

    # Encode text using Huffman codes
    encoded_text = "".join(codes[char] for char in text)

    return encoded_text

# Function to calculate probability of occurrence for each character
def calculate_probabilities(text):
    total_characters = len(text)
    freq_map = Counter(text)
    probabilities = {char: freq / total_characters for char, freq in freq_map.items()}
    return probabilities

# Function to calculate entropy
def calculate_entropy(probabilities):
    entropy = -sum(prob * math.log2(prob) for prob in probabilities.values())
    return entropy

# Function to calculate average length of compressed text
def calculate_average_length(compressed_text, probabilities):
    # Check if the compressed_text is a dictionary
    if isinstance(compressed_text, dict):
        # Iterate over the probabilities and access compressed_text using keys
        average_length = sum(prob * len(str(compressed_text[char])) for char, prob in probabilities.items())
    else:
        # Handle the case when compressed_text is not a dictionary
        # Assuming it's a string, calculate its length
        average_length = len(compressed_text)
    return average_length

# Calculate compression efficiency
def calculate_efficiency(original_size, compressed_size):
    efficiency = (1 - compressed_size / original_size) * 100
    return efficiency

# Update the compress_text function
def compress_text():
    global compression_technique_var
    
    # Get the input text from the text widget
    input_text = input_text_widget.get("1.0", "end-1c")
    
    # Get the selected compression technique
    selected_technique = compression_technique_var.get()
    
    # Perform compression using the selected technique
    if selected_technique == "Run-length Encoding":
        compressed_text = run_length_encoding(input_text)
    elif selected_technique == "Huffman Encoding":
        compressed_text = huffman_encoding(input_text)
    elif selected_technique == "Arithmetic Encoding":
        compressed_text = arithmetic_encoding(input_text)
    elif selected_technique == "Golomb Encoding":
        compressed_text = golomb_encoding(input_text)
    elif selected_technique == "LZW Encoding":
        compressed_text = lzw_encoding(input_text)
    else:
        compressed_text = ""
    
    # Calculate compression ratio
    compression_ratio = calculate_compression_ratio(input_text, compressed_text)
    
    # Calculate probabilities of occurrence for each character
    probabilities = calculate_probabilities(input_text)
    
    # Calculate entropy
    entropy = calculate_entropy(probabilities)
    
    # Calculate average length
    average_length = calculate_average_length(compressed_text, probabilities)
    
    # Calculate compression efficiency
    original_size = len(input_text) * 8  # Assuming each character is 8 bits
    compressed_size = len(compressed_text)
    efficiency = calculate_efficiency(original_size, compressed_size)
    
    # Update the result text widgets
    compressed_text_widget.config(state="normal")
    compressed_text_widget.delete("1.0", tk.END)
    compressed_text_widget.insert(tk.END, compressed_text)
    compressed_text_widget.config(state="disabled")
    
    bits_before_encoding_widget.config(text=str(original_size))
    bits_after_encoding_widget.config(text=str(compressed_size))
    compression_ratio_widget.config(text=f"{compression_ratio:.2f}%")
    entropy_widget.config(text=f"{entropy:.2f}")
    average_length_widget.config(text=f"{average_length:.2f} bits")
    efficiency_widget.config(text=f"{efficiency:.2f}%")

# Create the main window
root = tk.Tk()
root.title("Lossless Data Compression Techniques")

# Create a text widget for input
input_text_widget = tk.Text(root, height=5, width=30)
input_text_widget.pack(pady=10)

# Create a label and option menu for compression technique selection
compression_techniques = [
    "Run-length Encoding",
    "Huffman Encoding",
    "Arithmetic Encoding",
    "Golomb Encoding",
    "LZW Encoding"
]
compression_technique_var = tk.StringVar(root)
compression_technique_var.set(compression_techniques[0])  # Default value
compression_label = tk.Label(root, text="Select Compression Technique:")
compression_label.pack()
compression_option_menu = tk.OptionMenu(root, compression_technique_var, *compression_techniques)
compression_option_menu.pack()

# Create a button to trigger compression
compress_button = tk.Button(root, text="Compress", command=compress_text)
compress_button.pack()

# Create a label and text widget for displaying compressed text
compressed_text_label = tk.Label(root, text="Compressed Text:")
compressed_text_label.pack()
compressed_text_widget = tk.Text(root, height=5, width=30, state="disabled")
compressed_text_widget.pack(pady=10)

# Create labels to display compression-related information
bits_before_encoding_label = tk.Label(root, text="Bits Before Encoding:")
bits_before_encoding_label.pack()
bits_before_encoding_widget = tk.Label(root, text="")
bits_before_encoding_widget.pack()

bits_after_encoding_label = tk.Label(root, text="Bits After Encoding:")
bits_after_encoding_label.pack()
bits_after_encoding_widget = tk.Label(root, text="")
bits_after_encoding_widget.pack()

compression_ratio_label = tk.Label(root, text="Compression Ratio (%):")
compression_ratio_label.pack()
compression_ratio_widget = tk.Label(root, text="")
compression_ratio_widget.pack()

entropy_label = tk.Label(root, text="Entropy:")
entropy_label.pack()
entropy_widget = tk.Label(root, text="")
entropy_widget.pack()

average_length_label = tk.Label(root, text="Average Length:")
average_length_label.pack()
average_length_widget = tk.Label(root, text="")
average_length_widget.pack()

efficiency_label = tk.Label(root, text="Efficiency:")
efficiency_label.pack()
efficiency_widget = tk.Label(root, text="")
efficiency_widget.pack()

# Run the GUI
root.mainloop()
